<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2023TeamB/blob/main/Kopie_von_ANTICOVIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! git clone https://github.com/Fuenfgeld/DatamanagementAndArchiving

fatal: destination path 'DatamanagementAndArchiving' already exists and is not an empty directory.


In [6]:
import pandas as pd
from functools import reduce
import sqlite3
from sqlite3 import Error
import csv

In [7]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [8]:
material_path = "/content/DatamanagementAndArchiving/SynteticPatientData"

In [9]:
!ls "/content/DatamanagementAndArchiving/SynteticPatientData"

allergy  breast_cancer	    covid19	lung_cancer		    README.md
asthma	 colorectal_cancer  dermatitis	metabolic_syndrome_disease  source_dbs


In [10]:
## type of patients
# patient_type = "allergy"
# patient_type = "asthma"
# patient_type = "breast_cancer"
# patient_type = "metabolic_syndrome_disease"
# patient_type = "colorectal_cancer"
patient_type = "covid19"
# patient_type = "dermatitis"
# patient_type = "lung_cancer"

In [11]:
# source database path
DB_FILE_PATH = f"{material_path}/source_dbs/source_{patient_type}_test.db"

# folder contains csv files
csv_path = f"{material_path}/{patient_type}/"

DB_FILE_PATH, csv_path

('/content/DatamanagementAndArchiving/SynteticPatientData/source_dbs/source_covid19_test.db',
 '/content/DatamanagementAndArchiving/SynteticPatientData/covid19/')

In [12]:
# dictionary key: name of table, values: sql query to create table
sql_table = {}

In [13]:
# sql query to create disease table
sql_table["disease"] = """CREATE TABLE IF NOT EXISTS disease (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                           
                       );"""

In [14]:
#  sql query to create patients table
sql_table["patients"] = """CREATE TABLE IF NOT EXISTS patients (
                           Id STRING PRIMARY KEY,
                           BIRTHDATE DATE,
                           DEATHDATE DATE,
                           SSN STRING,
                           DRIVERS STRING,
                           PASSPORT STRING,
                           PREFIX STRING,
                           FIRST STRING,
                           LAST STRING,
                           SUFFIX STRING,
                           MAIDEN STRING,
                           MARITAL STRING,
                           RACE STRING,
                           ETHNICITY STRING,
                           GENDER STRING,
                           BIRTHPLACE STRING,
                           ADDRESS STRING,
                           CITY STRING,
                           STATE STRING,
                           COUNTRY STRING,
                           ZIP STRING,
                           LAT INTEGER,
                           LON INTEGER,
                           HEALTHCARE_EXPENSES INTEGER,
                           HEALTHCARE_COVERAGE INTEGER
                       );"""
      

In [15]:
# sql query to create encounters table
sql_table["encounters"] = """CREATE TABLE IF NOT EXISTS encounters (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ORGANIZATIONS STRING,
                           PROVIDER STRING,
                           PAYER STRING,
                           ENCOUNTERCLASS STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_ENCOUNTER_COST INTEGER,
                           TOTAL_CLAIM_COST INTEGER,
                           PAYER_COVERAGE INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                       );"""


In [16]:
# sql query to create careplans table
sql_table["careplans"] = """CREATE TABLE IF NOT EXISTS careplans (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""

In [17]:
# sql query to create conditions table
sql_table["conditions"] = """CREATE TABLE IF NOT EXISTS conditions (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""

In [18]:
# sql query to create medications table
sql_table["medications"] = """CREATE TABLE IF NOT EXISTS medications (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           PAYER STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           PAYER_COVERAGE INTEGER,
                           DISPENSES INTEGER,
                           TOTALCOST INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           
                       );"""

In [19]:
# # sql query to create procedures table
sql_table["procedures"]= """CREATE TABLE IF NOT EXISTS procedures (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""


In [20]:

# # sql query to create observations table
sql_table["observations"]= """CREATE TABLE IF NOT EXISTS observations (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           VALUE STRING,
                           UNITS STRING,
                           TYPE STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""

In [21]:
# sql query to create devices table
sql_table["devices"] = """CREATE TABLE IF NOT EXISTS devices (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           UDI STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
                           
                       );"""


In [22]:
# sql query to create Imaging_studies table
sql_table["imaging_studies"] = """CREATE TABLE IF NOT EXISTS imaging_studies (
                           Id STRING PRIMARY KEY,
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           BODYSITE_CODE STRING,
                           BODYSITE_DESCRIPTION STRING,
                           MODALITY_CODE STRING,
                           MODALITY_DESCRIPTION STRING,
                           SOP_CODE STRING,
                           SOP_DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""

In [23]:
# sql query to create Immunizations table
sql_table["immunizations"]= """CREATE TABLE IF NOT EXISTS immunizations(
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       );"""

In [24]:
db_file = DB_FILE_PATH

In [25]:
def connect_to_db(db_file):
    """
    Connect to an SQlite database, if db file does not exist it will be created
    :param db_file: absolute or relative path of db file
    :return: sqlite3 connection
    """
    sqlite3_conn = None

    try:
        sqlite3_conn = sqlite3.connect(db_file)
        return sqlite3_conn

    except Error as err:
        print(err)

        if sqlite3_conn is not None:
            sqlite3_conn.close()


def insert_values_to_table(cursor, table_name, csv_file_path):

    """
    Open a csv file, store its content in a list excluding header and insert the data from the list to db table
    :param table_name: table name in the database to insert the data into
    :param csv_file_path: path of the csv file to process
    :return: None
    """

    # Read CSV file content
    values_to_insert = open_csv_file(csv_file_path)

    # Insert to table
    if len(values_to_insert) > 0:
        column_names, column_numbers = get_column_names_from_db_table(cursor, table_name)

        values_str = '?,' * column_numbers
        # print("*****", values_str, column_names, column_numbers)
        values_str = values_str[:-1]

        sql_query = 'INSERT OR REPLACE INTO ' + table_name + '(' + column_names + ') VALUES (' + values_str + ')'


        cursor.executemany(sql_query, values_to_insert)
        # conn.commit()

        print('SQL insert process finished')
    else:
        print('Nothing to insert')

        # conn.close()

    # else:
        # print('Connection to database failed')


def open_csv_file(csv_file_path):
    """
    Open and read data from a csv file without headers (skipping the first row)
    :param csv_file_path: path of the csv file to process
    :return: a list with the csv content
    """
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)

        data = list()
        for row in reader:
            data.append(row)

        return data


def get_column_names_from_db_table(sql_cursor, table_name):
    """
    Scrape the column names from a database table to a list and convert to a comma separated string, count the number
    of columns in a database table
    :param sql_cursor: sqlite cursor
    :param table_name: table name to get the column names from
    :return: a comma separated string with column names, an integer with number of columns
    """

    table_column_names = 'PRAGMA table_info(' + table_name + ');'
    sql_cursor.execute(table_column_names)
    table_column_names = sql_cursor.fetchall()

    column_count = len(table_column_names)

    column_names = list()

    for name in table_column_names:
        column_names.append(name[1])

    return ', '.join(column_names), column_count



In [26]:
# list of tables
sql_table.keys()

dict_keys(['disease', 'patients', 'encounters', 'careplans', 'conditions', 'medications', 'procedures', 'observations', 'devices', 'imaging_studies', 'immunizations'])

In [27]:
# # remove all tables in a db
# for name in sql_table.keys():
#   cursor.execute(f"drop table if exists {name}")


In [40]:
# import csv into db
conn = connect_to_db(DB_FILE_PATH)
if conn is not None:
        cursor = conn.cursor()
        for name in sql_table.keys():
          csv_file = csv_path + name + ".csv"
          print(name, csv_file)
          
          cursor.execute(sql_table[name])

          insert_values_to_table(cursor, name, csv_file)
          conn.commit()
else:
        print('Connection to database failed')


disease /content/DatamanagementAndArchiving/SynteticPatientData/covid19/disease.csv
Nothing to insert
patients /content/DatamanagementAndArchiving/SynteticPatientData/covid19/patients.csv
SQL insert process finished
encounters /content/DatamanagementAndArchiving/SynteticPatientData/covid19/encounters.csv
SQL insert process finished
careplans /content/DatamanagementAndArchiving/SynteticPatientData/covid19/careplans.csv
SQL insert process finished
conditions /content/DatamanagementAndArchiving/SynteticPatientData/covid19/conditions.csv
SQL insert process finished
medications /content/DatamanagementAndArchiving/SynteticPatientData/covid19/medications.csv
SQL insert process finished
procedures /content/DatamanagementAndArchiving/SynteticPatientData/covid19/procedures.csv
SQL insert process finished
observations /content/DatamanagementAndArchiving/SynteticPatientData/covid19/observations.csv
SQL insert process finished
devices /content/DatamanagementAndArchiving/SynteticPatientData/covid19/

In [29]:
## ex, access to a csv file
 #pd.read_csv(f"{csv_path}disease.csv").head()

In [42]:
# List of tables in database 
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('disease',), ('patients',), ('encounters',), ('careplans',), ('conditions',), ('medications',), ('procedures',), ('observations',), ('devices',), ('imaging_studies',), ('immunizations',)]


In [ ]:
## check data in a table
cursor.execute("SELECT * FROM patients;")
rows = cursor.fetchall()
for row in rows:
  print(row)

In [34]:
conn.commit()

In [35]:
#conn.close()
